In [37]:
# ייבוא ספריות
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, log_loss
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
# ייבוא ספריות
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report, log_loss

In [29]:
# === שלב 1: קריאת הנתונים ===
# קריאת הקובץ players.csv לתוך DataFrame
df = pd.read_csv("players.csv")

# === שלב 2: טיפול בעמודות ===
# מחיקת עמודות שלא רלוונטיות למודל: שם השחקן ועמודת שנת העונה
df = df.drop(columns=['PLAYER', 'season year'])
df.head()

,Unnamed: 0,SEASON,AGE,POS,MP,FG,FGA,FG%,3P,3PA,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,0,0,28,PG,8.8,0.6,2.8,0.214,0.0,1.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,23,SG,25.9,2.1,7.7,0.275,1.0,4.8,...,3.8,15.5,-0.3,0.2,-0.1,-0.014,-6.8,-0.6,-7.5,-0.3
2,2,0,24,SG,1.0,0.0,0.0,0.000,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,-0.017,-7.4,-2.3,-9.7,0.0
3,3,0,24,SF,12.3,1.4,3.4,0.419,0.7,2.1,...,13.4,14.5,-0.1,0.3,0.3,0.027,-4.2,-1.2,-5.4,-0.4
4,4,0,20,SF,6.0,0.8,2.3,0.333,0.0,0.8,...,0.0,17.9,0.0,0.0,0.0,-0.031,-4.7,-1.7,-6.4,0.0


In [30]:

# קידוד משתנה קטגוריאלי - POS (עמדת השחקן): המרה לערכים מספריים
df['POS'] = LabelEncoder().fit_transform(df['POS'])
df.head()


,Unnamed: 0,SEASON,AGE,POS,MP,FG,FGA,FG%,3P,3PA,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,0,0,28,2,8.8,0.6,2.8,0.214,0.0,1.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,23,6,25.9,2.1,7.7,0.275,1.0,4.8,...,3.8,15.5,-0.3,0.2,-0.1,-0.014,-6.8,-0.6,-7.5,-0.3
2,2,0,24,6,1.0,0.0,0.0,0.000,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,-0.017,-7.4,-2.3,-9.7,0.0
3,3,0,24,4,12.3,1.4,3.4,0.419,0.7,2.1,...,13.4,14.5,-0.1,0.3,0.3,0.027,-4.2,-1.2,-5.4,-0.4
4,4,0,20,4,6.0,0.8,2.3,0.333,0.0,0.8,...,0.0,17.9,0.0,0.0,0.0,-0.031,-4.7,-1.7,-6.4,0.0


In [31]:
# === יצירת Target: האם שחקן קיבל הארכת חוזה ===
# חשוב מאוד: תחליף '10_day_contract' בשם העמודה שמייצגת את ה-Target שלך
y = df['SEASON']  # זו העמודה שמסמלת אם השחקן קיבל הארכת חוזה
y.head()


0    0
1    1
2    0
3    0
4    0
Name: SEASON, dtype: int64

In [32]:

# המשתנים המסבירים - כל שאר העמודות חוץ מה-Target
X = df.drop(columns=['SEASON'])
X.head()


,Unnamed: 0,AGE,POS,MP,FG,FGA,FG%,3P,3PA,3P%,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,0,28,2,8.8,0.6,2.8,0.214,0.0,1.4,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,23,6,25.9,2.1,7.7,0.275,1.0,4.8,0.209,...,3.8,15.5,-0.3,0.2,-0.1,-0.014,-6.8,-0.6,-7.5,-0.3
2,2,24,6,1.0,0.0,0.0,0.000,0.0,0.0,0.000,...,NaN,0.0,0.0,0.0,0.0,-0.017,-7.4,-2.3,-9.7,0.0
3,3,24,4,12.3,1.4,3.4,0.419,0.7,2.1,0.329,...,13.4,14.5,-0.1,0.3,0.3,0.027,-4.2,-1.2,-5.4,-0.4
4,4,20,4,6.0,0.8,2.3,0.333,0.0,0.8,0.000,...,0.0,17.9,0.0,0.0,0.0,-0.031,-4.7,-1.7,-6.4,0.0


In [33]:
# === טיפול בערכים חסרים (Missing Values) ===
# נחליף ערכים חסרים ב-0. אפשר לשפר את זה בהמשך עם שיטות יותר מתקדמות
X = X.fillna(0)
X.head()


,Unnamed: 0,AGE,POS,MP,FG,FGA,FG%,3P,3PA,3P%,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,0,28,2,8.8,0.6,2.8,0.214,0.0,1.4,0.000,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
1,1,23,6,25.9,2.1,7.7,0.275,1.0,4.8,0.209,...,3.8,15.5,-0.3,0.2,-0.1,-0.014,-6.8,-0.6,-7.5,-0.3
2,2,24,6,1.0,0.0,0.0,0.000,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,-0.017,-7.4,-2.3,-9.7,0.0
3,3,24,4,12.3,1.4,3.4,0.419,0.7,2.1,0.329,...,13.4,14.5,-0.1,0.3,0.3,0.027,-4.2,-1.2,-5.4,-0.4
4,4,20,4,6.0,0.8,2.3,0.333,0.0,0.8,0.000,...,0.0,17.9,0.0,0.0,0.0,-0.031,-4.7,-1.7,-6.4,0.0


In [34]:
# === נרמול הנתונים (Standardization) ===
# פעולה זו עוזרת למודלים (בעיקר רשתות נוירונים) ללמוד בצורה טובה יותר
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [19]:
# === חלוקת הנתונים ל-Train/Test (80% לאימון, 20% לבדיקה) ===
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [20]:
# === שלב 3: מודל קלאסי עם sklearn ===
# נתחיל עם Random Forest - מודל פשוט וחזק
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)  # אימון המודל על סט האימון

RandomForestClassifier(random_state=42)

In [21]:
# חיזוי על סט הבדיקה והדפסת דוח ביצועים
y_pred_rf = rf.predict(X_test)
print("\n=== תוצאות Random Forest ===")
print(classification_report(y_test, y_pred_rf))


=== תוצאות Random Forest ===
              precision    recall  f1-score   support

           0       0.73      0.95      0.83        83
           1       0.83      0.41      0.55        49

    accuracy                           0.75       132
   macro avg       0.78      0.68      0.69       132
weighted avg       0.77      0.75      0.72       132



In [35]:
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)

# חיזוי תוצאות
y_pred_logreg = logreg.predict(X_test)
y_proba_logreg = logreg.predict_proba(X_test)

# דוח ביצועים
print("\n=== תוצאות Logistic Regression ===")
print(classification_report(y_test, y_pred_logreg))

# חישוב פונקציית ההפסד (log loss)
loss_logreg = log_loss(y_test, y_proba_logreg)
print(f"Log Loss: {loss_logreg:.4f}")


=== תוצאות Logistic Regression ===
              precision    recall  f1-score   support

           0       0.75      0.90      0.82        83
           1       0.75      0.49      0.59        49

    accuracy                           0.75       132
   macro avg       0.75      0.70      0.71       132
weighted avg       0.75      0.75      0.74       132

Log Loss: 0.5991


In [38]:
# === יצירת המודלים ===
logreg = LogisticRegression(max_iter=1000, random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# VotingClassifier: ממוצע הסתברויות של המודלים
voting_clf = VotingClassifier(estimators=[
    ('lr', logreg),
    ('rf', rf)
], voting='soft')  # soft = ממוצע הסתברויות

# אימון VotingClassifier
voting_clf.fit(X_train, y_train)

# חיזוי והערכת הביצועים
y_pred_voting = voting_clf.predict(X_test)
y_proba_voting = voting_clf.predict_proba(X_test)

print("\n=== תוצאות Voting Classifier (Logistic + Random Forest) ===")
print(classification_report(y_test, y_pred_voting))

loss_voting = log_loss(y_test, y_proba_voting)
print(f"Log Loss: {loss_voting:.4f}")

# === סיכום קצר ===
print("\n✅ Voting Classifier מוכן! שילבנו Logistic Regression + Random Forest בהצלחה 🚀")


=== תוצאות Voting Classifier (Logistic + Random Forest) ===
              precision    recall  f1-score   support

           0       0.73      0.90      0.81        83
           1       0.72      0.43      0.54        49

    accuracy                           0.73       132
   macro avg       0.73      0.67      0.67       132
weighted avg       0.73      0.73      0.71       132

Log Loss: 0.5192

✅ Voting Classifier מוכן! שילבנו Logistic Regression + Random Forest בהצלחה 🚀


In [22]:
# === שלב 4: מודל רשת נוירונים עם Keras ===
# בניית רשת פשוטה: שכבת קלט, שתי שכבות חבויות, שכבת פלט
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))  # שכבת קלט עם 64 נוירונים
model.add(Dense(32, activation='relu'))  # שכבה חבויה
model.add(Dense(1, activation='sigmoid'))  # שכבת פלט - סיגמואיד כי זו בעיה בינארית

c:\Repos\BGU\Home_Asigments\NI\scratch_detection_assignment\my_work\venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
# קומפילציה של המודל: פונקציית עלות + אופטימייזר
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [24]:
# אימון המודל
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test), verbose=0)


In [25]:
# הערכת המודל על סט הבדיקה
loss, accuracy = model.evaluate(X_test, y_test)
print("\n=== תוצאות Neural Network ===")
print(f"Accuracy: {accuracy:.4f}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7449 - loss: 1.1255 

=== תוצאות Neural Network ===
Accuracy: 0.7424


In [26]:
# === סיכום ===
print("\n✅ סיום שלב ראשון - המודלים מוכנים 🚀")


✅ סיום שלב ראשון - המודלים מוכנים 🚀
